<a href="https://colab.research.google.com/github/JacobDowns/glamacles_test/blob/main/test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import firedrake as fd
from model import SpecFO
import numpy as np
import matplotlib.pyplot as plt


model = SpecFO()
data = np.load('data/data.npy')

/home/firedrake/firedrake/src/firedrake/firedrake/interpolation.py:385: FutureWarning: The use of `interpolate` to perform the numerical interpolation is deprecated.
This feature will be removed very shortly.

Instead, import `interpolate` from the `firedrake.__future__` module to update
the interpolation's behaviour to return the symbolic `ufl.Interpolate` object associated
with this interpolation.

You can then assemble the resulting object to get the interpolated quantity
of interest. For example,

```
from firedrake.__future__ import interpolate
...

assemble(interpolate(expr, V))
```

Alternatively, you can also perform other symbolic operations on the interpolation operator, such as taking
the derivative, and then assemble the resulting form.

  warnings.warn("""The use of `interpolate` to perform the numerical interpolation is deprecated.
/home/firedrake/firedrake/src/firedrake/firedrake/interpolation.py:385: FutureWarning: The use of `interpolate` to perform the numerical inter

In [8]:
import torch
from torch.utils.data import TensorDataset, DataLoader

# Create input features
B = data[:,0,:,:]
H = data[:,1,:,:]
S = B+H
S_x = np.gradient(S, axis=2)
S_y = np.gradient(S, axis=1)
X = np.stack([H, S_x, S_y], axis=1)

# Output features
Y = data[:,2:,:,:]

# Normalization
X /= X.std(axis=(0,2,3))[np.newaxis,:,np.newaxis,np.newaxis]
Y /= Y.std(axis=(0,2,3))[np.newaxis,:,np.newaxis,np.newaxis]

# Create datsets
N_train = 2000
X = torch.tensor(X, dtype=torch.float32)
Y = torch.tensor(Y, dtype=torch.float32)

train_dataset = TensorDataset(X[0:N_train], Y[0:N_train])
validation_dataset = TensorDataset(X[N_train:], Y[N_train:])

# Data loaders
train_loader = DataLoader(train_dataset, batch_size=1, shuffle=True)
validation_loader = DataLoader(validation_dataset, batch_size=1, shuffle=True)

In [18]:
from modulus.models.fno import FNO

# Fourier neural operator model
fno = FNO(
    in_channels=3,
    out_channels=4,
    decoder_layers=1,
    decoder_layer_size=32,
    dimension=2,
    latent_channels=32,
    num_fno_layers=5,
    num_fno_modes=16,
    padding=9,
).cuda()

optimizer = torch.optim.Adam(fno.parameters(), lr=1e-4)

# Number of epochs
epochs = 100


for epoch in range(epochs):
    avg_loss = 0.
    num = 0

    fno.train()
    for i, (x,y) in enumerate(train_loader):
        x = x.cuda()
        y = y.cuda()
        optimizer.zero_grad()
        y_fno = fno(x)
        loss = torch.mean(torch.sqrt((y - y_fno)**2 + 1e-10))
        loss.backward()
        optimizer.step()
        avg_loss += loss.item()
        num += 1

    print('avg_loss', avg_loss / num)

    # Validation 
    if epoch % 10 == 0:
        with torch.no_grad():
            validation_loss = 0.
            for i, (x,y) in enumerate(validation_loader):
                x = x.cuda()
                y = y.cuda()
                y_fno = fno(x)
                loss = torch.mean(torch.sqrt((y - y_fno)**2 + 1e-10))
                validation_loss += loss.item()
                num += 1
            print('validation_loss', validation_loss / num)
   


avg_loss 0.1303151687363861
validation_loss 0.009572764673549682
avg_loss 0.0493670450047357
avg_loss 0.04523443771712482
avg_loss 0.04259942564630183
avg_loss 0.04234569265443133
avg_loss 0.03803399472287856
avg_loss 0.03601163247518707
avg_loss 0.03469529780256562
avg_loss 0.031499064227507916
avg_loss 0.031643184785963965
avg_loss 0.029117395719862542
validation_loss 0.0051994941167999055
avg_loss 0.0282710539966356
avg_loss 0.028204611211607698
avg_loss 0.026507551262067865
avg_loss 0.025426495507010257
avg_loss 0.025717643451702316
avg_loss 0.024982524751132586
avg_loss 0.024419724831241184
avg_loss 0.022871167279750807
avg_loss 0.021612393893505215
avg_loss 0.0222585211553087
validation_loss 0.003259282735432498
avg_loss 0.021710637648386183
avg_loss 0.020549417398055082
avg_loss 0.01981383199649281
avg_loss 0.02117550425507943
avg_loss 0.0196045850829687
avg_loss 0.019336133171775145
avg_loss 0.018947884078137578
avg_loss 0.0187406107953866
avg_loss 0.01935881278567831
avg_loss 